In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.style.use('ggplot')

warnings.filterwarnings("ignore")

#import graphviz 
from sklearn.datasets import make_classification

%matplotlib inline
plt.style.use('ggplot')

In [3]:
url= "https://fbref.com/es/equipos/206d90db/2023-2024/partidos/c12/schedule/Resultados-y-partidos-de-Barcelona-La-Liga"
tables = pd.read_html(url)
df = tables[0]

In [4]:
df=df[['Fecha','Día','Sedes', 'Resultado', 'GF', 'GC','Adversario','Formación','Árbitro']]

In [5]:
df = df.dropna(subset=['Resultado'])

In [6]:
df['Día'] = df['Día'].replace({'Sáb':6,'Dom':7,'Lun':1,'Mar':2,'Mié':3,'Jue':4,'Vie':5})
df['Resultado'] = df['Resultado'].replace({'V':3,'E':2,'D':1})
df['Sedes'] = df['Sedes'].replace({'Local':1,'Visitante':0})

In [7]:
df['Anfitrion']='Barcelona'

### Estadisticas básicas

In [8]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_basic = tables[0]

In [9]:
df_basic=df_basic[['RL','Equipo','PG','PE', 'PP','GF','GC', 'xG', 'xGA','Últimos 5','Máximo Goleador del Equipo']]

In [10]:
df_basic['Máximo Goleador del Equipo'] = df_basic['Máximo Goleador del Equipo'].apply(lambda x: int(re.search(r'\b(\d+)\b', x).group(1)) if re.search(r'\b(\d+)\b', x) else None)

df_basic['Últimos 5'] = df_basic['Últimos 5'].apply(lambda resultados: sum([3 if resultado == 'PG' else (1 if resultado == 'PE' else 0) for resultado in resultados.split()]))


### Estadisticas de Ofensiva

In [11]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_ataque = tables[2]

In [12]:
df_ataque = df_ataque.drop(["Tiempo Jugado","Expectativa",'Por 90 Minutos'],axis=1)

In [13]:
df_ataque.columns = df_ataque.columns.droplevel(level=0)

In [14]:
df_ataque=df_ataque[['Equipo', 'Edad', 'Pos.','Ass','TPint', 'PrgC', 'PrgP']]

##### Disparos

In [15]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_disparos = tables[8]

In [16]:
df_disparos.columns=df_disparos.columns.droplevel(level=0)

In [17]:
df_disparos=df_disparos[['Equipo', '% de TT','Dist']]

In [18]:
df_ataque = pd.merge(df_ataque, df_disparos, left_on='Equipo', right_on='Equipo', how='left')

##### Pases

In [19]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_pases = tables[10]

In [20]:
df_pases = df_pases.drop(["Cortos","Medios",'Largos','Expectativa'],axis=1)

In [21]:
df_pases.columns=df_pases.columns.droplevel(level=0)

In [22]:
df_pases=df_pases[['Equipo', '% Cmp','Dist. tot.']]

In [23]:
df_ataque = pd.merge(df_ataque, df_pases, left_on='Equipo', right_on='Equipo', how='left')

### Estadisticas de defensa

In [24]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_porteria = tables[4]

In [25]:
df_porteria = df_porteria.drop(["Tiempo Jugado","Tiros penales"],axis=1)

In [26]:
df_porteria.columns = df_porteria.columns.droplevel(level=0)

In [27]:
df_porteria=df_porteria[['Equipo', 'GC', 'DaPC', 'Salvadas','PaC']]

In [28]:
url= "https://fbref.com/es/comps/12/Estadisticas-de-La-Liga"
tables = pd.read_html(url)
df_defensa = tables[16]

In [29]:
df_defensa = df_defensa.drop(['Desafíos'],axis=1)

In [30]:
df_defensa.columns=df_defensa.columns.droplevel(level=0)

In [31]:
df_defensa=df_defensa[['Equipo', 'TklG','Int','Err']]

In [32]:
df_final = pd.merge(df_ataque, df_defensa, left_on='Equipo', right_on='Equipo', how='left')

In [33]:
df_final = pd.merge(df_final, df_basic, left_on='Equipo', right_on='Equipo', how='left')

In [34]:
df_opp=df_final.copy()
df_tm=df_final.copy()

In [35]:
columns_to_rename = ['Edad', 'Pos.', 'Ass', 'TPint', 'PrgC', 'PrgP', '% de TT',
       'Dist', '% Cmp', 'Dist. tot.', 'TklG', 'Int', 'Err', 'RL', 'PG', 'PE',
       'PP', 'GF', 'GC', 'xG', 'xGA', 'Últimos 5',
       'Máximo Goleador del Equipo']
new_column_names_tm = [f"{col}(tm)" for col in columns_to_rename]
df_tm.rename(columns=dict(zip(columns_to_rename, new_column_names_tm)), inplace=True)

In [36]:
columns_to_rename = ['Edad', 'Pos.', 'Ass', 'TPint', 'PrgC', 'PrgP', '% de TT',
       'Dist', '% Cmp', 'Dist. tot.', 'TklG', 'Int', 'Err', 'RL', 'PG', 'PE',
       'PP', 'GF', 'GC', 'xG', 'xGA', 'Últimos 5',
       'Máximo Goleador del Equipo']
new_column_names_opp = [f"{col}(opp)" for col in columns_to_rename]
df_opp.rename(columns=dict(zip(columns_to_rename, new_column_names_opp)), inplace=True)

In [37]:
df = pd.merge(df, df_opp, left_on='Adversario', right_on='Equipo', how='left')

In [38]:
df = pd.merge(df, df_tm, left_on='Anfitrion', right_on='Equipo', how='left')

In [39]:
df=df.drop(['Equipo_x','Equipo_y'],axis=1)

In [40]:
df.head()

,Fecha,Día,Sedes,Resultado,GF,GC,Adversario,Formación,Árbitro,Anfitrion,...,RL(tm),PG(tm),PE(tm),PP(tm),GF(tm),GC(tm),xG(tm),xGA(tm),Últimos 5(tm),Máximo Goleador del Equipo(tm)
0,2023-08-13,7,0,2,0,0,Getafe,4-2-3-1,César Soto,Barcelona,...,1,4,0,0,13,3,11.1,2.9,12,4
1,2023-08-20,7,1,3,2,0,Cádiz,3-4-3,Alejandro Muñíz,Barcelona,...,1,4,0,0,13,3,11.1,2.9,12,4
2,2023-08-27,7,0,3,4,3,Villarreal,4-3-3,Alejandro Hernández,Barcelona,...,1,4,0,0,13,3,11.1,2.9,12,4
3,2023-09-03,7,0,3,2,1,Osasuna,4-3-3,Miguel Ángel Ortiz Arias,Barcelona,...,1,4,0,0,13,3,11.1,2.9,12,4
4,2023-09-16,6,1,3,5,0,Betis,4-3-3,José Sánchez,Barcelona,...,1,4,0,0,13,3,11.1,2.9,12,4


### Exportamos el archivo

In [41]:
archivo_destino = "laliga.csv"

# Escribe el dataframe en el archivo de destino
df.to_excel(archivo_destino)

ModuleNotFoundError: No module named 'openpyxl'

In [40]:
X=df[['Día','Sedes', 'GF', 'GC','Edad(opp)','Pos.(opp)', 'Ass(opp)', 'TPint(opp)', 
      'PrgC(opp)', 'PrgP(opp)','% de TT(opp)', 'Dist(opp)', '% Cmp(opp)', 'Dist. tot.(opp)','TklG(opp)', 'Int(opp)', 
      'Err(opp)', 'RL(opp)', 'PG(opp)', 'PE(opp)','PP(opp)', 'GF(opp)', 'GC(opp)', 'xG(opp)', 'xGA(opp)','Últimos 5(opp)', 
      'Máximo Goleador del Equipo(opp)', 'Edad(tm)', 'Pos.(tm)', 'Ass(tm)', 'TPint(tm)', 'PrgC(tm)', 'PrgP(tm)',
      '% de TT(tm)', 'Dist(tm)', '% Cmp(tm)', 'Dist. tot.(tm)', 'TklG(tm)','Int(tm)', 'Err(tm)', 'RL(tm)', 'PG(tm)', 
      'PE(tm)', 'PP(tm)', 'GF(tm)','GC(tm)', 'xG(tm)', 'xGA(tm)', 'Últimos 5(tm)','Máximo Goleador del Equipo(tm)']]
y=df['Resultado']